# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [37]:
df = pd.read_csv(r"C:\Users\Sritha\Downloads\AviationData.csv", encoding='latin1')

df.head()
df.info()
df.isna().sum()
df.describe()
df.describe(include='object')

C:\Users\Sritha\AppData\Local\Temp\ipykernel_39320\3076951026.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Sritha\Downloads\AviationData.csv", encoding='latin1')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Amateur.Built,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
count,88889,88889,88889,88889,88837,88663,34382,34373,50132,52704,...,88787,81793,32023,12582,82697,16648,84397,61724,82505,75118
unique,87951,2,88863,14782,27758,219,25592,27156,10374,24870,...,2,12,31,3,26,13590,4,12,17074,2924
top,20001214X45071,Accident,ERA22LA103,2000-07-08,"ANCHORAGE, AK",United States,332739N,0112457W,NONE,Private,...,No,Reciprocating,091,NSCH,Personal,Pilot,VMC,Landing,Probable Cause,25-09-2020
freq,3,85015,2,25,434,82248,19,24,1488,240,...,80312,69530,18221,4474,49448,258,77303,15428,61754,17019


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [38]:
#make sure Event.Date is a datetime type
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

#filter Aircraft.Category to only airplanes
df = df[df['Aircraft.Category'] == 'Airplane']

#filter Amateur.Built to only professional builds
df = df[df['Amateur.Built'] == 'No']

#filter Event.Date to keep only events from 1983 onwards
df = df[df['Event.Date'].dt.year >= 1983]

# drop rows with missing Event.Date after conversion
df = df.dropna(subset=['Event.Date'])

#inspect the shape of the filtered dataset
print(df.shape)

(21447, 31)


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [39]:
df['Total.Passengers'] = (df['Total.Fatal.Injuries'].fillna(0) + df['Total.Serious.Injuries'].fillna(0) 
                          + df['Total.Minor.Injuries'].fillna(0) + df['Total.Uninjured'].fillna(0))

df['Serious.Fatal.Fraction'] = ((df['Total.Fatal.Injuries'].fillna(0) + df['Total.Serious.Injuries'].fillna(0)) /
    df['Total.Passengers'])

df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Passengers', 'Serious.Fatal.Fraction']].head()

,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Passengers,Serious.Fatal.Fraction
4149,NaN,NaN,588.0,0.0
4150,NaN,NaN,588.0,0.0
4171,1.0,1.0,2.0,1.0
4285,1.0,NaN,5.0,0.2
5957,NaN,NaN,289.0,0.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [40]:
#inspect unique values first
df['Aircraft.damage'].value_counts(dropna=False)

#fill missing values with 'UNKNOWN' and make lowercase for consistency
df['Aircraft.damage_clean'] = df['Aircraft.damage'].fillna('UNKNOWN').str.lower()

#create a new column 'Destroyed' using a simple for loop
destroyed_list = []  #empty list to store True/False

for damage in df['Aircraft.damage_clean']:
    if 'destroyed' in damage:
        destroyed_list.append(True)
    else:
        destroyed_list.append(False)

#add this list as a new column
df['Destroyed'] = destroyed_list

#check first few rows
df[['Aircraft.damage', 'Aircraft.damage_clean', 'Destroyed']].head(10)

,Aircraft.damage,Aircraft.damage_clean,Destroyed
4149,Minor,minor,False
4150,Minor,minor,False
4171,Destroyed,destroyed,True
4285,NaN,unknown,False
5957,Minor,minor,False
5960,Destroyed,destroyed,True
6669,NaN,unknown,False
6760,Minor,minor,False
6806,Substantial,substantial,False
7084,Substantial,substantial,False


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [41]:
#drop rows with missing Make
df = df.dropna(subset=['Make'])

#standardize capitalization (title case)
df['Make_clean'] = df['Make'].str.title()

#keep only Makes with at least 50 accidents
make_counts = df['Make_clean'].value_counts()
makes_to_keep = make_counts[make_counts >= 50].index

df = df[df['Make_clean'].isin(makes_to_keep)]

#check result
df['Make_clean'].value_counts()

Make_clean
Cessna                            7146
Piper                             3989
Beech                             1431
Boeing                            1264
Mooney                             363
Airbus                             243
Cirrus Design Corp                 220
Bellanca                           219
Air Tractor Inc                    219
Maule                              215
Air Tractor                        206
Aeronca                            200
Champion                           158
Embraer                            153
Grumman                            147
Luscombe                           141
Cirrus                             137
Stinson                            129
Mcdonnell Douglas                  108
North American                     106
Dehavilland                         95
Taylorcraft                         93
Aero Commander                      90
Aviat Aircraft Inc                  76
Socata                              75
Diamond Aircra

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [42]:
#remove rows with NaN in Model
df = df.dropna(subset=['Model'])

#standardize Model names (title case)
df['Model_clean'] = df['Model'].str.title()

#inspect counts of models per make
model_counts = df.groupby(['Make_clean', 'Model_clean']).size().reset_index(name='count')
model_counts.head(10)

#see if Model labels are unique across Makes
duplicate_models = df.groupby('Model_clean')['Make_clean'].nunique()
duplicate_models = duplicate_models[duplicate_models > 1]
print("Models that exist for multiple Makes:\n", duplicate_models)

#create a unique identifier for plane type
#if a Model exists across multiple Makes, combine Make and Model
df['Plane_Type'] = df['Make_clean'] + " " + df['Model_clean']

Models that exist for multiple Makes:
 Model_clean
100      2
112      2
112A     2
140      2
190      2
        ..
Sr-20    2
Sr-22    2
Sr20     2
Sr22     2
Sr22T    2
Name: Make_clean, Length: 118, dtype: int64


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [44]:
#cleaning Other Columns

#cClean column names: remove spaces/periods, standardize capitalization
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('.', '_').str.title()
print("Cleaned Columns:\n", df.columns.tolist())

#engine Type
print("\nUnique Engine Types:\n", df['Engine_Type'].value_counts(dropna=False))
# Cleaning tasks:
# - Standardize capitalization
# - Replace obvious placeholders like 'UNKNOWN', '', or 'Unkown' with NaN
df['Engine_Type'] = df['Engine_Type'].replace(['UNKNOWN', '', 'Unkown'], pd.NA).str.title()

#weather Condition
print("\nUnique Weather Conditions:\n", df['Weather_Condition'].value_counts(dropna=False))
# Cleaning tasks:
# - Standardize capitalization
# - Replace placeholders like 'UNK', 'Unknown', '' with NaN
df['Weather_Condition'] = df['Weather_Condition'].replace(['UNK', 'Unknown', ''], pd.NA).str.title()

#number of Engines
print("\nSummary of Number of Engines:\n", df['Number_Of_Engines'].describe())
# Cleaning tasks:
# - Remove negative or zero values (they don't make sense)
df.loc[df['Number_Of_Engines'] <= 0, 'Number_Of_Engines'] = pd.NA

# Cleaning tasks:
# - Standardize capitalization
# - Replace placeholders like 'UNK', 'Unknown', '' with NaN
df['Purpose_Of_Flight'] = df['Purpose_Of_Flight'].replace(['UNK', 'Unknown', ''], pd.NA).str.title()

#broad Phase of Flight
print("\nUnique Broad Phases of Flight:\n", df['Broad_Phase_Of_Flight'].value_counts(dropna=False))
# Cleaning tasks:
# - Standardize capitalization
# - Replace placeholders like 'UNK', 'Unknown', '' with NaN
df['Broad_Phase_Of_Flight'] = df['Broad_Phase_Of_Flight'].replace(['UNK', 'Unknown', ''], pd.NA).str.title()


Cleaned Columns:
 ['Event_Id', 'Investigation_Type', 'Accident_Number', 'Event_Date', 'Location', 'Country', 'Latitude', 'Longitude', 'Airport_Code', 'Airport_Name', 'Injury_Severity', 'Aircraft_Damage', 'Aircraft_Category', 'Registration_Number', 'Make', 'Model', 'Amateur_Built', 'Number_Of_Engines', 'Engine_Type', 'Far_Description', 'Schedule', 'Purpose_Of_Flight', 'Air_Carrier', 'Total_Fatal_Injuries', 'Total_Serious_Injuries', 'Total_Minor_Injuries', 'Total_Uninjured', 'Weather_Condition', 'Broad_Phase_Of_Flight', 'Report_Status', 'Publication_Date', 'Total_Passengers', 'Serious_Fatal_Fraction', 'Aircraft_Damage_Clean', 'Destroyed', 'Make_Clean', 'Model_Clean', 'Plane_Type']

Unique Engine Types:
 Engine_Type
Reciprocating      12835
NaN                 3214
Turbo Prop           931
Turbo Fan            701
Unknown              105
Turbo Jet             71
Geared Turbofan       12
Turbo Shaft            9
UNK                    1
Name: count, dtype: int64

Unique Weather Conditions

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [45]:
#inspect the number of non-null values per column
non_null_counts = df.count()
print("Non-null counts per column:\n", non_null_counts)

#keep only columns with more than 20,000 non-null values
columns_to_keep = non_null_counts[non_null_counts > 20000].index
df = df[columns_to_keep]

#check the resulting dataframe
print("\nColumns kept:\n", df.columns)
print("\nShape of cleaned dataframe:", df.shape)

Non-null counts per column:
 Event_Id                  17879
Investigation_Type        17879
Accident_Number           17879
Event_Date                17879
Location                  17875
Country                   17878
Latitude                  15981
Longitude                 15978
Airport_Code              11648
Airport_Name              11754
Injury_Severity           17162
Aircraft_Damage           16815
Aircraft_Category         17879
Registration_Number       17715
Make                      17879
Model                     17879
Amateur_Built             17879
Number_Of_Engines         15785
Engine_Type               14665
Far_Description           17535
Schedule                   2139
Purpose_Of_Flight         14529
Air_Carrier                8448
Total_Fatal_Injuries      15493
Total_Serious_Injuries    15421
Total_Minor_Injuries      15656
Total_Uninjured           17292
Weather_Condition         15386
Broad_Phase_Of_Flight      2441
Report_Status             14094
Publication

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [53]:

print(df.shape)
print(df.head())


df.to_csv(r"C:\Users\Sritha\Downloads\AviationData_Cleaned.csv", index=False)

(17879, 0)
Empty DataFrame
Columns: []
Index: [4150, 4171, 4285, 6760, 6806]
